In [77]:
#%%writefile ./from_csv_to_jsonl.py
import csv
import re
#import cloudstorage as gcs
from google.cloud import bigquery
client = bigquery.Client()

In [94]:
from google.cloud import bigquery_storage
bqstorageclient = bigquery_storage.BigQueryReadClient()
bqclient = bigquery.Client()
# Download query results.
query_string = """
select
concat(cast(id as string), ".txt") as text_id, 
replace(body_text, ",", "") as body_text
from `transactiondata.dw_pl_ml_freshdesk.sample2`
"""

dataframe = (
    bqclient.query(query_string)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
print(dataframe.head())

           text_id                                          body_text
0  60059311026.txt  Hi there. Do you have any similar bikes I coul...
1  60061198109.txt  Hello Michael\r\n\r\nThank you for your quick ...
2  60061443900.txt  Hi Twaha\r\n\r\nThank you for the prompt respo...
3  60060502289.txt  Many thanks for this. I appreciate it. Got a l...
4  60060482833.txt                                         \r\n\r\n> 


In [78]:
%%bigquery sample
select
concat(cast(id as string), '.txt') as text_id, 
replace(replace(replace(body_text, ',', ''), '\n', ''), '\r', '') as body_text
from `transactiondata.dw_pl_ml_freshdesk.sample2`

In [46]:
for iD, text in zip(list(sample['text_id']), list(sample['body_text'])):
    text_id = 'sample/' + iD
    with open(text_id, "w") as id_text:
        id_text.write(text)

In [56]:
#!gsutil cp -r sample gs://dw_freshdesk/  #copy the answers over to the bucket

In [59]:
%%bigquery sample
select concat('gs://dw_freshdesk/sample/', concat(cast(id as string), '.txt')) as content, 
"text/plain" as mimeType
FROM `transactiondata.dw_pl_ml_freshdesk.sample2`

In [60]:
with open("sample.jsonl", "w") as text_file:
    s= sample.to_json(orient='records', lines=True)
    t = re.sub('[^\u0000-\u007f]', '',  s)
    text_file.write(t) 

    #{"content": "gs://sourcebucket/datasets/texts/source_text.txt", "mimeType": "text/plain"}


Then I download the above jsonl file and I upload it into the same bucket as above and then I head over to Vertex AI and I cick on the "batch prediction" button; giving the addresses as required. 